## Graph Intelligence: Google ADK + Neo4j MCP
This notebook demonstrates how to build a generative AI agent that can autonomously interact with a Neo4j Graph Database using the Model Context Protocol (MCP).

### 1. Environment Setup & Configuration
First, we install the necessary libraries.

• google-adk: The framework for building agents.  
• neo4j: The driver for database connectivity.  
• mcp: The protocol layer for tool communication.

In [ ]:
# Install core dependencies
%pip install -q google-adk neo4j mcp requests

#### Imports

In [ ]:
import os
import base64
import subprocess
import time
import requests
import warnings
from getpass import getpass
from neo4j import GraphDatabase
from google.genai import types
from google.adk.agents.llm_agent import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts.in_memory_artifact_service import InMemoryArtifactService
from google.adk.tools.mcp_tool import McpToolset, StreamableHTTPConnectionParams
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from google.adk.tools.function_tool import FunctionTool
from mcp import StdioServerParameters

warnings.filterwarnings("ignore", category=UserWarning)

#### LLM Configuration

In [2]:
os.environ["GOOGLE_API_KEY"] = getpass("Enter Google API Key: ")
os.environ["GEMINI_MODEL"] = "gemini-flash-latest"

#### DB configuration

In [3]:
os.environ["NEO4J_URI"] = "neo4j+s://demo.neo4jlabs.com"
os.environ["NEO4J_DATABASE"] = "companies"

### 2. Installing the Neo4j MCP Server
The MCP server acts as the "bridge" between the LLM and your database. It translates natural language intent into schema-aware Cypher queries.

In [5]:
# Fetch and install the latest Linux binary for the Neo4j MCP server
release = requests.get("https://api.github.com/repos/neo4j/mcp/releases/latest").json()
version = release["tag_name"]
print(f"Installing Neo4j MCP version: {version}")

!wget -q https://github.com/neo4j/mcp/releases/download/{version}/neo4j-mcp_Linux_x86_64.tar.gz
!tar -xzf neo4j-mcp_Linux_x86_64.tar.gz
!sudo chmod +x neo4j-mcp
!sudo mv neo4j-mcp /usr/local/bin/
!rm -f neo4j-mcp_Linux_x86_64.tar.gz

print("Verification:")
!neo4j-mcp -v

Installing Neo4j MCP version: v1.4.0
Verification:
neo4j-mcp version: v1.4.0


### 3. Toolset Configuration
You have two ways to connect the ADK to the MCP server.

#### Option A: Standard I/O (Recommended for local/internal scripts)
This launches the MCP server as a subprocess of your Python script.

In [ ]:
# Connection details for the public Neo4j Sandbox. Only use this for stdio connection testing. For HTTP connection, credentials will be passed in the header.
os.environ["NEO4J_USERNAME"] = "companies"
os.environ["NEO4J_PASSWORD"] = "companies"

mcp_tools = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command='neo4j-mcp', 
            args=[], 
            env={
                "NEO4J_URI": os.environ["NEO4J_URI"],
                "NEO4J_USERNAME": os.environ["NEO4J_USERNAME"],
                "NEO4J_PASSWORD": os.environ["NEO4J_PASSWORD"],
                "NEO4J_DATABASE": os.environ["NEO4J_DATABASE"],
                "NEO4J_READ_ONLY": "true"
            }
        )
    )
)

#### Option B: HTTP (Recommended for distributed or hosted environments)
This starts a persistent background process.  
**Note:** Authentication is handled via a Base64 encoded header rather than environment variables in the transport layer. Refer official [documentation](https://neo4j.com/docs/mcp/current/installation/)

In [4]:
# Encode credentials for HTTP Header
credentials = base64.b64encode(b"companies:companies").decode()
PORT = "8443"

# Start background server
process = subprocess.Popen(
    [
        "neo4j-mcp",
        "--neo4j-uri", os.environ["NEO4J_URI"],
        "--neo4j-transport-mode", "http",
        "--neo4j-http-port", PORT,
        "--neo4j-database", os.environ["NEO4J_DATABASE"],
    ],
    stdout=subprocess.PIPE, 
    stderr=subprocess.PIPE, 
    text=True
)

time.sleep(3) # Warm-up time

# Check if the process started
poll = process.poll()
if poll is not None:
    stdout, stderr = process.communicate()
    print(f"Server failed to start (Exit Code: {poll})")
    print(f"STDOUT: {stdout}")
    print(f"STDERR: {stderr}")
else:
    print(f"Server started successfully on port {PORT}")


Server started successfully on port 8443


In [5]:
#Defining the tool with HTTP connection parameters, including the Authorization header
mcp_tools = McpToolset(
    connection_params=StreamableHTTPConnectionParams(
        url=f"http://localhost:{PORT}/mcp",
        headers={"Authorization": f"Basic {credentials}"}
        ),
)

### 4. Agent Construction
We wrap the ADK LlmAgent creation in an async function. The agent is provided with the mcp_tools we defined above.

In [6]:
async def create_agent(model, name, prompt, tools_list):
    """
    Constructs an LLM Agent with specialized tools.
    The agent uses the system prompt to understand it should favor 'get-schema' 
    before attempting complex queries.
    """
    return LlmAgent(
        model=model,
        name=name,
        instruction=prompt,
        tools=tools_list
    )

system_prompt = """
You are a graph database assistant. Your job is to answer user questions by querying Neo4j.

Always run 'get-schema' first if you are unfamiliar with the graph structure.
Use Cypher queries to retrieve data.
If the data is not found, state that clearly.
"""

mcp_agent = await create_agent(
    model=os.environ["GEMINI_MODEL"], 
    name="neo4j_explorer", 
    prompt=system_prompt, 
    tools_list=[mcp_tools]
)

### 5. Execution Engine
The Runner orchestrates the conversation, handling tool calls (the "back-and-forth" between Gemini and Neo4j) automatically.

In [7]:
async def ask_graph(agent, query):
    session_service = InMemorySessionService()
    artifacts_service = InMemoryArtifactService()

    session = await session_service.create_session(state={}, app_name="neo4j_app", user_id="user_1")
    content = types.Content(role='user', parts=[types.Part(text=query)])

    runner = Runner(
        app_name="neo4j_app",
        agent=agent,
        artifact_service=artifacts_service,
        session_service=session_service,
    )

    print(f"Processing: {query}")
    try:
        events_async = runner.run_async(
            session_id=session.id, 
            user_id=session.user_id, 
            new_message=content
        )

        async for event in events_async:
            if hasattr(event, 'content') and event.content:
                for part in event.content.parts:
                    if part.text:
                        print(f"\nResult: {part.text}")
    except Exception as e:
        print(f"Error: {e}")

# Example Query
await ask_graph(mcp_agent, "How many people are in the database?")

Processing: How many people are in the database?



Result: There are 8,064 people in the database.


### 6. Custom tools
Beyond using existing MCP servers, you can also implement your own custom tools and add them directly to the agent. This allows you to create specialized functionality tailored to your specific use case. Custom tools can be implemented using the @tool decorator, which turns any function into a tool the agent can invoke.

Here, we use GraphDatabase from the neo4j package to establish a connection to our database and build a tool that queries investment relationships, giving you more control over the query logic.

In [12]:
async def get_investments(company: str) -> str:
    """
    Returns the investments by a company by name. 
    Returns list of investment ids, names and types.
    """
    query = """
    MATCH (o:Organization)-[:HAS_INVESTOR]->(i)
    WHERE o.name = $company
    RETURN i.id as id, i.name as name, head(labels(i)) as type
    """
    try:
        driver = GraphDatabase.driver(os.environ["NEO4J_URI"], auth=(os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"]), database=os.environ["NEO4J_DATABASE"])
        records, _, _ = driver.execute_query(
            query, 
            company=company
        )
        results = [record.data() for record in records]
        return json.dumps(results, indent=2)
    except Exception as e:
        return f"Error fetching investments: {str(e)}"


Transforming a Python function into a tool is a straightforward way to integrate custom logic into your agents. When you assign a function to an agent’s tools list, the framework automatically wraps it as a FunctionTool.

In [14]:
custom_investment_tool = FunctionTool(get_investments)

In [15]:
custom_tools = [mcp_tools, custom_investment_tool]
custom_agent_prompt = "You are a helpful assistant with access to a Neo4j graph database containing company data. Use the available tools to query the database and answer questions."
custom_agent_name = "Custom_neo4j_agent"
custom_agent = await create_agent(os.environ["GEMINI_MODEL"], custom_agent_name, custom_agent_prompt, custom_tools)

In [17]:
query = "Which companies did Google invest in?"
await ask_graph(custom_agent, query)

Processing: Which companies did Google invest in?

Result: Google has invested in the following companies:

*   **Ionic Security**
*   **Avere Systems**
*   **FlexiDAO**
*   **Cloudflare**
*   **Trifacta**


# Summary

This notebook demonstrates how to use Google's Agent Development Kit (ADK) to build conversational agents that interact with a Neo4j graph database. The workflow includes:

- Connecting to the Neo4j MCP server for schema and Cypher query access.
- Defining and running an LLM agent with MCP tools.
- Implementing and integrating custom tools for specialized queries.
- Running agents to answer questions using both MCP and custom tools.